In [61]:
import mysql.connector
import sys
sys.path.append('../')
from transform import read_data, parse_data
from CN_config import apiKey, mysql_u, mysql_p
import pandas as pd
import datetime

## Read Data

In [53]:
def connect_base(user, passwd):
    mydb = mysql.connector.connect(user=user,
                                    passwd=passwd,
                                    host='msitapati.mysql.pythonanywhere-services.com',
                                    database='msitapati$crypto_news')
    print(f'Successfully connected to MySQL')
    return mydb

In [54]:
mydb = connect_base(mysql_u, mysql_p)

Successfully connected to MySQL


In [55]:
def show_tables(mydb):
    import mysql.connector
    mycursor = mydb.cursor()
    mycursor.execute("SHOW TABLES")
    print('Tables in current DB:')
    for x in mycursor:
        print(x)
    return mycursor



In [56]:
show_tables(mydb)

Tables in current DB:
('articles',)


In [57]:
def select_data(mydb):
    mycursor = mydb.cursor()
    mycursor.execute('SELECT * FROM articles')
    myresult = mycursor.fetchall()
    return pd.DataFrame(myresult, columns=['publishedAt', 'source', 'source_id', 'author', 'title', 'description', 'content', 'url'])

In [58]:
df = select_data(mydb)

In [59]:
df.head()

,publishedAt,source,source_id,author,title,description,content,url
0,2020-04-25 15:14:11,Forbes,forbes,"Biser Dimitrov, Contributor, \n Biser Dimitrov...",Big Four Accounting Firm Wants To Empower Ente...,Using the public Ethereum blockchain for enter...,"Paul Brody, Global Blockchain Innovation Leade...",https://www.forbes.com/sites/biserdimitrov/202...
1,2020-04-26 09:57:47,Bloomberg,bloomberg,None,Swiss 'Crypto Valley' In Talks for More Financ...,None,None,https://www.bloomberg.com/news/articles/2020-0...
2,2020-04-26 15:10:25,Forbes,forbes,"Crypto Confidential, Forbes Staff, \n Crypto C...",Bitcoin Hits April High; Crypto Portfolios Get...,"The number of deposits of exactly $1,200 more ...",Get Forbes' top crypto and blockchain stories ...,https://www.forbes.com/sites/cryptoconfidentia...
3,2020-04-27 01:15:47,CoinDesk,coindesk,Noelle Acheson,Crypto Long & Short: How Oil Going Negative Co...,The economic repercussions of oil going negati...,This will always be remembered as the week whe...,https://www.coindesk.com/oil-going-negative-bi...
4,2020-04-27 06:46:25,Forbes,forbes,"Luke Fitzpatrick, Contributor, \n Luke Fitzpat...",Bitcoin Halving: A New Class Of Bitcoin Millio...,The third bitcoin halving could be a major cat...,The photo shows a physical imitation of a Bitc...,https://www.forbes.com/sites/lukefitzpatrick/2...


In [62]:
df.to_pickle(f'{datetime.datetime.now().strftime("%y%m%d-%H%M%S")}-df.pkl')

In [63]:
unpickled_df = pd.read_pickle('200603-225635-df.pkl')

In [64]:
unpickled_df.head()

,publishedAt,source,source_id,author,title,description,content,url
0,2020-04-25 15:14:11,Forbes,forbes,"Biser Dimitrov, Contributor, \n Biser Dimitrov...",Big Four Accounting Firm Wants To Empower Ente...,Using the public Ethereum blockchain for enter...,"Paul Brody, Global Blockchain Innovation Leade...",https://www.forbes.com/sites/biserdimitrov/202...
1,2020-04-26 09:57:47,Bloomberg,bloomberg,None,Swiss 'Crypto Valley' In Talks for More Financ...,None,None,https://www.bloomberg.com/news/articles/2020-0...
2,2020-04-26 15:10:25,Forbes,forbes,"Crypto Confidential, Forbes Staff, \n Crypto C...",Bitcoin Hits April High; Crypto Portfolios Get...,"The number of deposits of exactly $1,200 more ...",Get Forbes' top crypto and blockchain stories ...,https://www.forbes.com/sites/cryptoconfidentia...
3,2020-04-27 01:15:47,CoinDesk,coindesk,Noelle Acheson,Crypto Long & Short: How Oil Going Negative Co...,The economic repercussions of oil going negati...,This will always be remembered as the week whe...,https://www.coindesk.com/oil-going-negative-bi...
4,2020-04-27 06:46:25,Forbes,forbes,"Luke Fitzpatrick, Contributor, \n Luke Fitzpat...",Bitcoin Halving: A New Class Of Bitcoin Millio...,The third bitcoin halving could be a major cat...,The photo shows a physical imitation of a Bitc...,https://www.forbes.com/sites/lukefitzpatrick/2...


In [8]:
#df['source'] = df['source'].astype('category')

In [9]:
#df['source_id'] = df['source_id'].astype('category')

In [10]:
#df['author'] = df['author'].astype('category')

In [11]:
len(df.loc[df['title'].str.contains('crypto', case=False)])

159

In [12]:
len(df.loc[df['title'].str.contains('bitcoin', case=False)])

279

In [13]:
len(df.loc[df['title'].str.contains('ethereum', case=False)])

51

## Work on DF copy

In [14]:
df_key = df.copy()

In [15]:
df_key['author'] = df['author'].str.split(',', expand=True)[0]

In [16]:
df_key['author'] = df_key['author'].str.replace('Cointelegraph By ','')

In [17]:
df_key['author'] = df_key['author'].str.replace(')','')

In [18]:
df_key['author'] = df_key['author'].str.split('(', expand=True)[0]

In [19]:
df_key['crypto'] = df_key['title'].str.contains('crypto', case=False)

In [20]:
df_key['bitcoin'] = df_key['title'].str.contains('bitcoin', case=False)

In [21]:
df_key['ethereum'] = df_key['title'].str.contains('ethereum', case=False)

## Title Sentiment Analysis

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [23]:
sentences = df['title']

In [24]:
analyzer = SentimentIntensityAnalyzer()

In [25]:
df_key['t_comp'] = pd.Series()
df_key['t_neg'] = pd.Series()
df_key['t_pos'] = pd.Series()
df_key['t_neut'] = pd.Series()
for i,v in enumerate(df_key['title']):
    vs = analyzer.polarity_scores(v)
    df_key['t_comp'][i] = vs['compound']
    df_key['t_neg'][i] = vs['neg']
    df_key['t_pos'][i] = vs['pos']
    df_key['t_neut'][i] = vs['neu']

/usr/lib/python3.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/IPython/kernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/lib/python3.7/site-packages/IPython/kernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cavea

In [26]:
df_vs = df_key[['publishedAt', 'source_id', 't_comp', 't_neg', 't_pos', 't_neut']]

In [27]:
df_vs = df_vs.set_index('publishedAt')

In [28]:
df_vs = df_vs.sort_index()

## Plotting

In [29]:
df_vs.head()

,source_id,t_comp,t_neg,t_pos,t_neut
publishedAt,,,,,
2020-04-25 15:14:11,forbes,0.0000,0.000,0.000,1.000
2020-04-26 09:57:47,bloomberg,0.4549,0.000,0.271,0.729
2020-04-26 15:10:25,forbes,0.0000,0.000,0.000,1.000
2020-04-27 01:15:47,coindesk,-0.5719,0.209,0.000,0.791
2020-04-27 06:46:25,forbes,0.0000,0.000,0.000,1.000


In [30]:
df_plot = df_vs.resample('D').mean()

In [31]:
df_plot = df_plot.sort_index()

In [32]:
import pandas as pd
import numpy as np

# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

In [47]:
from bokeh.plotting import reset_output

# Use reset_output() between subsequent show() calls, as needed
reset_output()

In [34]:
source = ColumnDataSource(df_plot)

p = figure(x_axis_type='datetime', plot_width=800, plot_height=350)
p.line('publishedAt', 't_comp', source=source)

output_file("ts.html")
show(p)



In [49]:
source = ColumnDataSource(df_plot)

p = figure(x_axis_type='datetime', plot_width=800, plot_height=350)
p.vbar(x='publishedAt', width=1, top='t_comp', source=source)

output_file("ts.html")
show(p)


In [48]:
source = ColumnDataSource(df_plot)

p = figure(x_axis_type='datetime', plot_width=800, plot_height=350)
p.vbar(x='publishedAt', width=0.5, top='t_comp', source=source)

output_file("ts.html")
show(p)

In [51]:
from bokeh.io import output_file, show
from bokeh.plotting import figure

output_file("stacked.html")

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=data)
             #legend_label=years)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)